In [38]:
import os
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch
from dotenv import load_dotenv
load_dotenv("..\\.env")
api_key = os.getenv('SEARCH_INDEX_NAME')
print(api_key)
print("yay")

demo-gerardo-index
yay


In [39]:
embeddings = OpenAIEmbeddings(deployment="demo-search", chunk_size=1)

# Connect to Azure Cognitive Search
acs = AzureSearch(azure_search_endpoint=os.getenv('SEARCH_SERVICE_NAME'),
                 azure_search_key=os.getenv('SEARCH_API_KEY'),
                 index_name=os.getenv('SEARCH_INDEX_NAME'),
                 embedding_function=embeddings.embed_query)

In [30]:
# Verifying path is correct
# import pandas as pd
# pd.read_csv("wine-ratings.csv")

In [40]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader(file_path="wine-ratings.csv", encoding='utf-8')
documents = loader.load()

In [41]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

acs.add_documents(documents=docs)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-07-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 2 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Requests to the Embeddings_Create Operation under Azure OpenAI API version 2023-07-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 10 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


KeyboardInterrupt: 

In [42]:
docs = acs.similarity_search_with_relevance_scores(
    query="What is the best Cabernet Sauvignon wine in Napa Valley above 94 points",
    k=5,
)
print(docs[0][0].page_content)
print(dir(docs[0][0]))

IndexError: list index out of range

In [8]:
openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_type = "azure"
openai.api_version = "2023-10-01-preview" 
messages=[
    {"role": "system", "content": "Asisstant is a chatbot that helps you find the best wine for your taste."},
    {"role": "user", "content": "What is the best wine in Oregon above 92 points?"},
    {"role": "assistant", "content": docs[0][0].page_content}
]

response = openai.ChatCompletion.create(
    engine="demo-alfredo",
    messages=messages,
)
from pprint import pprint
pprint(response)
print(response['choices'][0]['message']['content'])

{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'message': {'content': 'I apologize for the confusion, but I '
                                     "don't have access to real-time wine "
                                     'ratings and reviews. It would be best to '
                                     'refer to professional wine rating '
                                     'websites or consult with a sommelier for '
                                     'specific recommendations on Oregon Pinot '
                                     'Noir wines above 94 points. They will '
                                     'have the most up-to-date and accurate '
                                     'information for you.',
                          'role': 'assistant'}}],
 'created': 1703696035,
 'id': 'chatcmpl-8aRSFAMFjRLXsPOGHajprdnUPTeuu',
 'model': 'gpt-35-turbo',
 'object': 'chat.completion',
 'usage': {'completion_tokens': 66,
           'prompt_tokens': 154,
